In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from Map import Map
from Tabu import Tabu
from TwoStage import TwoStage
from L_shaped import MasterProblem as mp
import matplotlib.pyplot as plt
import matplotlib.pylab as pl 
import matplotlib.cm as cm
import time
import networkx as nx
import matplotlib as mpl
from gurobipy import *
import csv

**Modelling Parameters**

In [3]:
rides = 3
bus = 2
scenarios = 10
MIPGap = 0.001
TimeLimit = 18000
probability = [0.7, 0.1, 0.15, 0.05]

**Defining the Map**

In [4]:
mappy = Map(rides, seed=200)

**Plotting Functions**

In [5]:
def plot_trend(ub,lb,base):
    x = list(range(len(ub)))
    figure = plt.figure()
    plt.plot(x,ub,label='upper-bound')
    plt.plot(x,[base]*len(ub),label='two-stage')
    plt.plot(x,lb,label='lower-bound')
    plt.annotate(int(ub[-1]),[x[-1],ub[-1]])
    plt.legend()
    plt.savefig('./figures/trend.png')
    plt.show()

colors = ['green','blue','yellow','red','pink'] 
def displaygraph(n, e, modname):
    for k in e.keys():
        for i, j in e[k]:
            plt.plot((n[i][0],n[j][0]),(n[i][1],n[j][1]), color=colors[k] ,marker='o', linewidth=2, linestyle='dashed')
    for i in n:
        plt.annotate(i,n[i],textcoords="offset points",xytext=(0,10),ha='center')
    plt.savefig('./figures/'+modname+'.png')
    plt.show()


#### Model formulation

In [6]:
def Mod1(aux=False):
    t1 = time.time()
    drpstw = mp(mappy,bus=bus,scenarios=scenarios, probability=probability)
    drpstw.initialize()
    drpstw.setMIP()
    drpstw.variables.th.obj=0
    if aux:
        drpstw.setLowerBound()
        for s in range(drpstw.scenarios):
            drpstw.submodel[s].model.params.MIPGap = MIPGap
#             drpstw.submodel[s].model.params.OutputFlag = 1
    else:
        drpstw.model.params.TimeLimit = TimeLimit
        drpstw.model.optimize()
    t1 = time.time() - t1
    return drpstw, t1

In [7]:
def Mod2(model=None):
    t2 = time.time()
    twosp = TwoStage(mappy, bus=bus, scenarios=scenarios, probability=probability)
    twosp.model.params.TimeLimit = TimeLimit
    if model is not None:
        for i,j,k in twosp.MP.variables.x.keys():
            twosp.MP.variables.x[i,j,k].start = model.variables.x[i,j,k].X
    twosp.optimize()
    
    master =sum(twosp.MP.variables.x[i, j, k].X * twosp.parameters.distance[i, j]
                     + twosp.MP.variables.h[i, j].X
                     # * twosp.MP.variables.w[i]
                     for i, j in twosp.parameters.edges for k in range(twosp.MP.bus)) + \
            sum(twosp.MP.variables.p_l[i].X for i in twosp.parameters.pickup)
    
    savings = (sum((1 / twosp.scenarios) * (twosp.parameters.distance[i, j] * (twosp.variables.xs[i, j, k, s].X -
                                                                               twosp.MP.variables.x[i, j, k].X)
                                             + (twosp.variables.hs[i, j, s].X - twosp.MP.variables.h[i, j].X))
                                             for i, j in twosp.parameters.edges
                                             for k in range(twosp.MP.bus) for s in twosp.parameters.xi) +
                                    sum((1 / twosp.scenarios) * (twosp.variables.p_ls[i, s].X - twosp.MP.variables.p_l[i].X)
                                             for i in twosp.parameters.pickup for s in twosp.parameters.xi))
    
    t2 = time.time() - t2
    return twosp, savings,master, t2

In [8]:
def Mod3():
    t3 = time.time()
    lshaped = mp(mappy, bus=bus, scenarios=scenarios, probability=probability)
    lshaped.initialize()
    lshaped.model.params.TimeLimit = TimeLimit
    lshaped.model.params.MIPGap = MIPGap
#     if lshaped.model.runtime < TimeLimit:
#         lshaped.model.params.MIPGap
    lshaped.optimize(skip_init=False)
    savings = sum((1/scenarios)*lshaped.submodel[i].model.ObjVal for i in lshaped.submodel.keys())
    t3 = time.time() - t3
    return lshaped, savings, t3

In [9]:
def Mod4(lshaped):
    t4 = time.time()
    if lshaped.submodel is not None:
        tabu = Tabu(lshaped,tabu_iterations=800, tabu_status=rides + 2, rtoptm=5, subset=20, tsp=False)
        tabu.tsp = True
    else:
        tabu = Tabu(lshaped,tabu_iterations=800, tabu_status=rides + 2, rtoptm=5, subset=20, tsp=True)
    tabu.tabuheuristic()
    t4 = time.time() - t4
    return tabu, t4

In [10]:
def Mod5():
    t3 = time.time()
    lshaped = mp(mappy, bus=bus, scenarios=scenarios, probability=probability)
    lshaped.initialize()
    lshaped.model.params.TimeLimit = TimeLimit
    lshaped.optimize(skip_init=True)
    savings = sum((1/scenarios)*lshaped.submodel[i].model.ObjVal for i in lshaped.submodel.keys())
    t3 = time.time() - t3
    return lshaped, savings, t3

### Test Block

In [ ]:
TimeLimit = 18000
tests = [(3,2),(4,2),(5,2),(6,2),(7,2),(9,3),(10,3),(11,3),(13,4),(15,4),(18,5),(20,5),(22,6),(24,6),(25,7),(27,8)]
scenarioslist = [10,20,30]

csv_file = open('./Reports/Report.csv', 'w', newline='')
write = csv.writer(csv_file, delimiter=',')
write.writerow(('Rides','Bus','Scenarios', 'DRPObj', 'DRPTime', 'TSPObj', 'TSPSave','TSPGap','TSPTime','LshapeNsObj','LshapeNsSave','LshapeNsGap','LshapeNsTime','LshapeObj','LshapeSave','LshapeGap','LshapeTime','TabuObj','TabuSave','TabuTime'))
csv_file.close()
for s in scenarioslist:
    scenarios = s
    for test in tests:
        
        rides, bus = test
        mappy = Map(rides, seed=200)

        #Darp-stw
        try:
            drpstw, t1 = Mod1()
            if drpstw.model.status in [2,9]:
                drpobj = drpstw.model.ObjVal
                drptime = t1
            else:
                drpobj = 'NA'
                drptime = t1
        except (GurobiError, AttributeError, KeyboardInterrupt):
            drpobj = 'Failed'
            drptime = 'Failed'
        #Two-Stage
        try:    
            twostage, savings, master, t1 = Mod2()
            if twostage.model.status in [2,9]:
                twostageobj = twostage.model.ObjVal
                twostagesave = savings
                twostagegap = twostage.model.MIPGap
                twostagetime = t1
            else:
                twostageobj = 'NA'
                twostagesave = 'NA'
                twostagegap = 'NA'
                twostagetime = t1
        except (GurobiError, AttributeError, KeyboardInterrupt):
            twostageobj = 'Failed'
            twostagesave = 'Failed'
            twostagegap = 'Failed'
            twostagetime = 'Failed'
        #L-shaped no initialisation
        try:    
            lshaped, apprxskip, t2 = Mod5()
            if lshaped.model.status in [2,9]:
                lshapedobjns = lshaped.model.ObjVal
                lshapedsavens = lshaped.variables.th.X
                lshapedgapns = lshaped.model.MIPGap
                lshapedtimens = t2
            else:
                lshapedobjns = 'NA'
                lshapedgapns = 'NA'
                lshapedtimens = t2
                lshapedsavens = 'NA'
        except (GurobiError, AttributeError, KeyboardInterrupt):
            lshapedobjns = 'Failed'
            lshapedsavens = 'Failed'
            lshapedgapns = 'Failed'
            lshapedtimens = 'Failed'
        #L-shaped
        try:    
            lshaped, apprx, t2 = Mod3()
            if lshaped.model.status in [2,9]:
                lshapedobj = lshaped.model.ObjVal
                lshapedsave = lshaped.variables.th.X
                lshapedgap = lshaped.model.MIPGap
                lshapedtime = t2
            else:
                lshapedobj = 'NA'
                lshapedgap = 'NA'
                lshapedtime = t2
                lshapedsave = 'NA'
        except (GurobiError, AttributeError, KeyboardInterrupt):
            lshapedobj = 'Failed'
            lshapedsave = 'Failed'
            lshapedgap = 'Failed'
            lshapedtime = 'Failed'
        #Tabu
        try:    
            tabu, t3 = Mod4(lshaped)
            tabuobj = tabu.best[-1]
            tabusave = tabu.best[-3]
            tabutime = t3
        except (GurobiError, AttributeError, KeyboardInterrupt):
            tabuobj = 'Failed'
            tabusave = 'Failed'
            tabutime = 'Failed'

        csv_file = open('./Reports/Report.csv', 'a', newline='')
        write = csv.writer(csv_file, delimiter=',')
        write.writerow((str(rides),
                        str(bus),
                        str(scenarios),
                        str(drpobj),
                        str(drptime),
                        str(twostageobj),
                        str(twostagesave),
                        str(twostagegap),
                        str(twostagetime),
                        str(lshapedobjns),
                        str(lshapedsavens),
                        str(lshapedgapns),
                        str(lshapedtimens),
                        str(lshapedobj),
                        str(lshapedsave),
                        str(lshapedgap),
                        str(lshapedtime),
                        str(tabuobj),
                        str(tabusave),
                        str(tabutime)))
        csv_file.close()
        

Using license file C:\Users\lavkeshg\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter Heuristics unchanged
   Value: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter TimeLimit to 18000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 350 rows, 151 columns and 1364 nonzeros
Model fingerprint: 0x366c866e
Variable types: 65 continuous, 86 integer (86 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+03]
Found heuristic solutio

     0     0 -1059.8051    0    8   50.27606 -1059.8051  2208%     -    0s
     0     0 -1059.8051    0   19   50.27606 -1059.8051  2208%     -    0s
     0     0 -1059.3699    0   22   50.27606 -1059.3699  2207%     -    0s
     0     0 -1059.3699    0   22   50.27606 -1059.3699  2207%     -    0s
     0     2 -1059.3699    0   22   50.27606 -1059.3699  2207%     -    0s
*  144    24              17    -390.8153777 -885.73938   127%  14.8    1s

Cutting planes:
  Gomory: 27
  Implied bound: 11
  Clique: 1
  MIR: 6
  Flow cover: 65
  Zero half: 21
  RLT: 3
  Relax-and-lift: 5
  Lazy constraints: 34

Explored 190 nodes (3131 simplex iterations) in 1.78 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: -390.815 50.2761 
No other solutions better than -390.815

Optimal solution found (tolerance 1.00e-04)
Best objective -3.908153777054e+02, best bound -3.908153777054e+02, gap 0.0000%

User-callback calls 632, time in user-callback 1.46 sec
Apprx = -390.815378
ObjVal

61.77156219972199 61.77156219972199
Neighborhood length: 2 (iteration 10)
61.77156219972199 61.77156219972199
Neighborhood length: 1 (iteration 11)
61.77156219972199 61.77156219972199
Neighborhood length: 0 (iteration 12)
61.77156219972199 61.77156219972199
Neighborhood length: 3 (iteration 13)
61.77156219972199 61.77156219972199
Neighborhood length: 2 (iteration 14)
61.77156219972199 61.77156219972199
Neighborhood length: 1 (iteration 15)
61.77156219972199 61.77156219972199
Neighborhood length: 0 (iteration 16)
61.77156219972199 61.77156219972199
Neighborhood length: 3 (iteration 17)
61.77156219972199 61.77156219972199
Neighborhood length: 2 (iteration 18)
61.77156219972199 61.77156219972199
Neighborhood length: 1 (iteration 19)
61.77156219972199 61.77156219972199
Neighborhood length: 0 (iteration 20)
61.77156219972199 61.77156219972199
Neighborhood length: 3 (iteration 21)
61.77156219972199 61.77156219972199
Neighborhood length: 2 (iteration 22)
61.77156219972199 61.77156219972199
Ne

     0     0   41.00583    0   33  196.86529   41.00583  79.2%     -    0s
     0     0   41.00583    0   33  196.86529   41.00583  79.2%     -    0s
     0     0   41.05333    0   37  196.86529   41.05333  79.1%     -    0s
     0     0   41.10245    0   37  196.86529   41.10245  79.1%     -    0s
     0     0   41.10688    0   38  196.86529   41.10688  79.1%     -    0s
     0     0   41.11055    0   39  196.86529   41.11055  79.1%     -    0s
     0     0   41.11055    0   39  196.86529   41.11055  79.1%     -    0s
     0     0   41.45141    0   42  196.86529   41.45141  78.9%     -    0s
     0     0   41.45271    0   42  196.86529   41.45271  78.9%     -    0s
     0     0   41.45545    0   43  196.86529   41.45545  78.9%     -    0s
     0     0   42.00000    0   26  196.86529   42.00000  78.7%     -    0s
     0     0   42.00000    0   26  196.86529   42.00000  78.7%     -    0s
     0     0   42.00100    0   39  196.86529   42.00100  78.7%     -    0s
     0     0   42.00184  

* 1814   409              23      47.8235070 -639.74596  1438%  12.9   11s
* 1890   422              23      46.0241179 -636.71116  1483%  12.8   12s
  2501   465  -81.74486   28    8   46.02412 -611.08728  1428%  12.6   15s
  3268   456     cutoff   35        46.02412 -554.25133  1304%  12.4   20s
  4395   250   -0.11754   36   16   46.02412 -422.37769  1018%  11.5   25s

Cutting planes:
  Gomory: 5
  Implied bound: 49
  Projected implied bound: 1
  Clique: 3
  MIR: 34
  Flow cover: 213
  Zero half: 68
  RLT: 9
  Relax-and-lift: 47
  Lazy constraints: 402

Explored 5122 nodes (56568 simplex iterations) in 27.73 seconds
Thread count was 8 (of 8 available processors)

Solution count 6: 46.0241 47.8235 99.2458 ... 325.141

Optimal solution found (tolerance 1.00e-04)
Best objective 4.602411793922e+01, best bound 4.602411793922e+01, gap 0.0000%

User-callback calls 10820, time in user-callback 25.07 sec
Apprx = 46.024118
ObjVal = 49.924729
203  Integrality Cuts integrated into the model
20

     0     0   28.46511    0   52  201.96533   28.46511  85.9%     -    1s
     0     0   28.59522    0   46  201.96533   28.59522  85.8%     -    2s
     0     0   28.59539    0   47  201.96533   28.59539  85.8%     -    2s
     0     0   28.60450    0   48  201.96533   28.60450  85.8%     -    2s
     0     0   28.60576    0   49  201.96533   28.60576  85.8%     -    2s
     0     0   28.60580    0   50  201.96533   28.60580  85.8%     -    2s
     0     0   28.61380    0   51  201.96533   28.61380  85.8%     -    2s
     0     0   28.61380    0   51  201.96533   28.61380  85.8%     -    2s
H    0     0                     168.3493782   28.61380  83.0%     -    2s
     0     2   29.22601    0   51  168.34938   29.22601  82.6%     -    2s
*   98    46              16      72.8902025   30.49043  58.2%  16.4    2s
*  145    44              10      52.9902025   30.49814  42.4%  14.4    3s
H  165    39                      47.8235070   30.49814  36.2%  13.7    3s
*  315    34             

74.8724032297175 74.8724032297175
Neighborhood length: 1 (iteration 94)
74.8724032297175 74.8724032297175
Neighborhood length: 0 (iteration 95)
74.8724032297175 74.8724032297175
Neighborhood length: 4 (iteration 96)
74.8724032297175 74.8724032297175
Neighborhood length: 3 (iteration 97)
74.8724032297175 74.8724032297175
Neighborhood length: 2 (iteration 98)
74.8724032297175 74.8724032297175
{0: 0-->3-->7-->9, 1: 0-->1-->4-->5-->8-->2-->6-->9, -1: 74.8724032297175, -2: 89.33974029879442, -3: -14.467337069076924}
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter Heuristics unchanged
   Value: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter TimeLimit to 18000.0
   Prev: 

     0     0   45.21882    0   65  103.50998   45.21882  56.3%     -    2s
     0     0   45.21909    0   63  103.50998   45.21909  56.3%     -    2s
     0     0   45.26426    0   58  103.50998   45.26426  56.3%     -    2s
     0     0   45.27106    0   60  103.50998   45.27106  56.3%     -    2s
     0     0   45.27333    0   59  103.50998   45.27333  56.3%     -    2s
     0     0   45.39011    0   57  103.50998   45.39011  56.1%     -    2s
     0     0   45.39011    0   59  103.50998   45.39011  56.1%     -    2s
     0     0   45.44015    0   58  103.50998   45.44015  56.1%     -    2s
     0     0   45.44161    0   61  103.50998   45.44161  56.1%     -    2s
     0     0   45.59073    0   60  103.50998   45.59073  56.0%     -    2s
H    0     0                      87.5099788   45.59073  47.9%     -    2s
     0     0   45.60616    0   59   87.50998   45.60616  47.9%     -    2s
     0     0   45.60971    0   62   87.50998   45.60971  47.9%     -    2s
     0     0   45.63250  

  RLT: 79
  Relax-and-lift: 58

Explored 9195 nodes (790138 simplex iterations) in 59.19 seconds
Thread count was 8 (of 8 available processors)

Solution count 10: 43.7625 43.781 43.811 ... 57.8356

Optimal solution found (tolerance 1.00e-04)
Best objective 4.376254310967e+01, best bound 4.376254310967e+01, gap 0.0000%
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter Heuristics unchanged
   Value: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter TimeLimit to 18000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with

In [ ]:
MIPGap = 0.001
tests = [(3,1),(4,1),(5,1),(6,2),(7,2),(8,2),(9,2)]
scenarioslist = [10,15,20]
for s in scenarioslist:
    scenarios = s
    for test in tests:
        rides, bus = test
        mappy = Map(rides)        
#         drpstw, t1 = Mod1()
#         objdrpstw = drpstw.model.ObjVal
        twosp, t2 = Mod2()
        objtwosp = twosp.model.ObjVal
        lshaped, t3 = Mod3() 
        objlshaped = lshaped.model.ObjVal
#         tabu, t4 = Mod4()
#         objtabu = tabu.best[-2]
        write = [
#             str(objdrpstw), str(t1),
            str(objtwosp), str(t2),
            str(objlshaped), str(t3),
#             str(objtabu), str(t4),
            str(scenarios),str(bus),str(rides)]
        with open('report.txt', 'a') as file:
            file.write('\n')
            for w in write:
                file.write(w)
                file.write(',')

### DARP-STW Model

#### Model Solution 

In [ ]:
drpstw, t1 = Mod1()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(drpstw.bus)}
for i, j in drpstw.parameters.edges:
    for k in range(drpstw.bus):
        if drpstw.variables.x[i,j,k].X > 0.5:
            if j != drpstw.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objdrpstw = drpstw.model.ObjVal
displaygraph(nodes, edges,'DARP-STW')
nodes,edges

### Two-Stage Stochastic Model

#### Model Solution

In [ ]:
twosp, savings, master, t2 = Mod2()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(twosp.bus)}
for i, j in twosp.parameters.edges:
    for k in range(twosp.bus):
        if twosp.MP.variables.x[i,j,k].X > 0.5:
            if j != twosp.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objtwosp = twosp.model.ObjVal
displaygraph(nodes, edges,'Two-Stage')
nodes,edges

### L-Shaped Method

#### Model Solution

In [ ]:
lshaped, apprx, t3 = Mod3()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.variables.x[i,j,k].X > 0.5:
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges,bounds

In [ ]:
plot_trend(lshaped.upperbounds[2:],lshaped.lowerbounds[2:],0)

In [ ]:
mostcancel = {i: 0 for i in lshaped.parameters.pickup}
for i in lshaped.parameters.pickup:
    for s in range(lshaped.scenarios):
        mostcancel[i] += 2 - lshaped.submodel[s].sim.alpha[i] - lshaped.submodel[s].sim.alpha[i+lshaped.parameters.rides]
mostcancel = sorted(mostcancel.items(), key=lambda x: (x[1]))
mostcancel

In [ ]:
lshaped.printsol()

In [ ]:
from _datetime import timedelta
str(timedelta(minutes=tabu.best[0][2].time))

In [ ]:
sum((1/scenarios)*lshaped.submodel[i].model.ObjVal for i in lshaped.submodel.keys())

### Tabu-Heuristic

#### Model Solution

In [ ]:
tabu, t4 = Mod4(lshaped)

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabu.bus:
    for i in tabu.best[k]:
        n = tabu.best[k][i]
        if i != tabu.model.last:
            edges[k].append((i,n.next.key))
objtabu = tabu.best[-2]
displaygraph(nodes, edges,'Tabu')    
nodes,edges, tabu.best

In [ ]:
# tabu.best[0] = LinkedList()
# [0,1,4,2,6,7,9,11]
for i in tabu.bestcandidate[0]:
    print(tabu.bestcandidate[0][i].bus)

In [ ]:
lshaped_skp, t5 = Mod5()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabutsp.bus:
    for i in tabutsp.best[k]:
        n = tabutsp.best[k][i]
        if i != tabutsp.model.last:
            edges[k].append((i,n.next.key))
objtabutsp = tabutsp.best[-2]
displaygraph(nodes, edges,'TabuTSP')    
nodes,edges, tabutsp.best

In [ ]:
t3,t4

In [ ]:
mappy.pickup_time

In [ ]:
# print(twosp.sim.alpha)
lshaped.printsol(lshaped.submodel)

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.submodel[3].variables.xs[i,j,k].X > 0.5:
            if i == 0 and j == 9:
                continue
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges

In [ ]:
drpstw